In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rexpi
import scipy.sparse
import time

# approximate action of the matrix exponential operator

compute $\exp(tA )b$ for a matrix $A\in\mathbb{C}^{k\times k}$ which spectrum resides on the imaginary axis, a time-step $t$ and a vector $b\in\mathbb{C}^k$.

The time-step $t$ can be understood as the frequency $\omega$, i.e., $t=\omega$.

In [2]:
n = 40
tol=1e-6
tolequi = 1e-3
w = rexpi.west(n,tol)
print("for n=%d and tol=%g, our error estimate suggests w=%f"%(n,tol,w))

r, info = rexpi.brib(w = w, n = n, tolequi = tolequi, info=1)
print("... compute unitary best approximant")
print("used %d iterations, error = %.2e, deviation = %.2e\n"%(info['iterations'],info['err'],info['dev']))

print("... compute coefficients for product and partial fraction form")
a0, aj, sj = r.getpartialfractioncoef(sym=True)
poles = sj

for n=40 and tol=1e-06, our error estimate suggests w=101.765352
... compute unitary best approximant
used 10 iterations, error = 9.83e-07, deviation = 6.87e-05

... compute coefficients for product and partial fraction form


## Tridiagonal matrix, errors in norm

In [3]:
# with direct solver, small dimension
inr = lambda x,y : np.vdot(x,y)
nrm = lambda x : np.linalg.norm(x)

print("r(A)b \u2248 exp(wA)b where r corresponds to the unitary best approximant of degree n=%d\n"%n)
print("use brib for n=%d, tol=%g, and w=%f"%(n,tol,w))

# define diagonal matrix with spectrum between -1 and 1
k=50
print("random starting vector b with \u2016b\u2016=1, dimension k=%d\n" %k)
u = np.random.rand(k)
u = u/nrm(u)
nrmb0 = 1

e1 = np.ones(k-1)
e = np.ones(k)
o = np.zeros(k)

# H is shifted Laplace operator with eigenvalues between -1 and 1
H = scipy.sparse.diags([0.5*e1,o,0.5*e1], [-1,0,1])

#reference solution
yref = scipy.sparse.linalg.expm_multiply(1j*w*H,u)

mv = lambda x : 1j*H.dot(x)
mvSaIiH = lambda s,x : (np.linalg.inv(1j*H.toarray() - s*np.eye(k))).dot(x)

yproductform = rexpi.evalr_product(mv, mvSaIiH, u, poles)
print("evaluate y=r(A)b in product form: \u2016y-exp(wA)b\u2016 = %.2e" % nrm(yref-yproductform))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(yproductform)-nrmb0))

yparfrac = rexpi.evalr_partialfraction(mvSaIiH, u, aj, sj)
print("evaluate y=r(A)b using partial fractions: \u2016y-exp(wA)b\u2016 = %.2e" % nrm(yref-yparfrac))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(yparfrac)-nrmb0))

#print("(error in l2 norm is bounded by scalar error since the spectrum of H is in [-1,1] and ||u||=1)")
#print("(computing r(iH)u via partial fraction is only accurate for r of small degree)")

r(A)b ≈ exp(wA)b where r corresponds to the unitary best approximant of degree n=40

use brib for n=40, tol=1e-06, and w=101.765352
random starting vector b with ‖b‖=1, dimension k=50

evaluate y=r(A)b in product form: ‖y-exp(wA)b‖ = 8.04e-07
error in norm, |‖y‖-‖b‖| = 6.66e-16

evaluate y=r(A)b using partial fractions: ‖y-exp(wA)b‖ = 8.04e-07
error in norm, |‖y‖-‖b‖| = 1.39e-11



### compare with Pade

In [4]:
npade = 70
print("r(wA)b \u2248 exp(wA)b where r corresponds to the diagonal Pad\u00E9 approximation to exp(z) of degree n=%d\n"%npade)
rpade = rexpi.pade(npade)


# evaluate r(iwH), argument has to be re-scaled by w
poles2 = rpade.getpoles()

mviHw = lambda x : 1j*w*H.dot(x)
mvSaIiHw = lambda s,x : (np.linalg.inv(1j*w*H.toarray() - s*np.eye(k))).dot(x)

ypadepoles = rexpi.evalr_product(mviHw, mvSaIiHw, u, poles2)
ap0, apj, spj = rpade.getpartialfractioncoef()

ypadeparfrac = rexpi.evalr_partialfraction(mvSaIiHw, u, apj, spj)

print("evaluate y=r(wA)b in product form: \u2016y-exp(wA)b\u2016 = %.2e" % nrm(yref-ypadepoles))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypadepoles)-nrmb0))
print("evaluate y=r(wA)b using partial fractions: \u2016y-exp(wA)b\u2016 = %.2e" % nrm(yref-ypadeparfrac))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypadeparfrac)-nrmb0))

r(wA)b ≈ exp(wA)b where r corresponds to the diagonal Padé approximation to exp(z) of degree n=70

evaluate y=r(wA)b in product form: ‖y-exp(wA)b‖ = 2.98e-03
error in norm, |‖y‖-‖b‖| = 1.55e-15

evaluate y=r(wA)b using partial fractions: ‖y-exp(wA)b‖ = 2.33e+15
error in norm, |‖y‖-‖b‖| = 2.33e+15



### compare with polynomial Chebyshev approximation

In [5]:
ncheb = 130
print("p(A)b \u2248 exp(wA)b where p corresponds to the polynomial Chebyshev approximation of degree n=%d\n"%ncheb)

# evaluate Chebyshev approximation using Clenshaw Algorithm

# evaluate with mv using Clenshaw Algorithm
mvH = lambda x : H.dot(x)
ypcheb = rexpi.chebyshev(mvH,w,u,ncheb)

print("error: \u2016p(A)b-exp(iwA)b\u2016 = %.2e" % nrm(yref-ypcheb))
print("error in norm, |\u2016p(A)b\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypcheb)-nrmb0))

p(A)b ≈ exp(wA)b where p corresponds to the polynomial Chebyshev approximation of degree n=130

error: ‖p(A)b-exp(iwA)b‖ = 1.97e-08
error in norm, |‖p(A)b‖-‖b‖| = 6.71e-09



### compare with rational interpolation at Chebyshev nodes

In [6]:
nrc = 48
print("r(A)b \u2248 exp(wA)b where r corresponds to the (n,n)-rational interpolant at 2n+1 Chebyshev nodes for degree n=%d\n"%nrc)
r2 = rexpi.riCheb(w, nrc)
a02, aj2, sj2 = r2.getpartialfractioncoef(sym=True)
poles2 = sj2
yprod2 = rexpi.evalr_product(mv, mvSaIiH, u, poles2)
print("evaluate y=r(A)b in product form: \u2016y-exp(iwA)b\u2016 = %.2e" % nrm(yref-yprod2))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(yprod2)-nrmb0))

ypfe2 = rexpi.evalr_partialfraction(mvSaIiH, u, aj2, sj2)
print("evaluate y=r(A)b using partial fractions: \u2016y-exp(iwA)b\u2016 = %.2e" % nrm(yref-ypfe2))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypfe2)-nrmb0))

r(A)b ≈ exp(wA)b where r corresponds to the (n,n)-rational interpolant at 2n+1 Chebyshev nodes for degree n=48

evaluate y=r(A)b in product form: ‖y-exp(iwA)b‖ = 5.31e-09
error in norm, |‖y‖-‖b‖| = 1.10e-14

evaluate y=r(A)b using partial fractions: ‖y-exp(iwA)b‖ = 6.82e-08
error in norm, |‖y‖-‖b‖| = 3.60e-08



## use unitary best approximant with Lapack tridiagonal solver
consider a large dimensinal for the matrix A and evaluate r(A)b using a tridiagonal solver

In [7]:
# with direct solver, small dimension
nrm = lambda x : np.linalg.norm(x)
print("r(A)b \u2248 exp(wA)b where r corresponds to the unitary best approximant of degree n=%d\n"%n)
print("use brib for n=%d, tol=%g, and w=%f"%(n,tol,w))
# define diagonal matrix with spectrum between -1 and 1
k=30000
print("random starting vector b with \u2016b\u2016=1, dimension k=%d\n" %k)
u = np.random.rand(k)
u = u/nrm(u)

e1 = np.ones(k-1,dtype=np.cdouble)
e = np.ones(k,dtype=np.cdouble)
o = np.zeros(k,dtype=np.cdouble)

# H is shifted Laplace operator with eigenvalues between -1 and 1
H = scipy.sparse.diags([0.5*e1,o,0.5*e1], [-1,0,1])

t1=time.time()
yref = scipy.sparse.linalg.expm_multiply(1j*w*H,u)
dt=time.time()-t1
print("time to compute reference solution %f seconds"%dt)

r(A)b ≈ exp(wA)b where r corresponds to the unitary best approximant of degree n=40

use brib for n=40, tol=1e-06, and w=101.765352
random starting vector b with ‖b‖=1, dimension k=30000

time to compute reference solution 0.546550 seconds


In [8]:
print("untiary best approximation with accuracy max|r(ix)-epx(iwx)| = %.2e\n"%info['err'])

# define sparse solver
mv = lambda x : 1j*H.dot(x)

# deep copies
dl2 = 1j*0.5*e1
dd2 = 1j*o
du2 = 1j*0.5*e1
def mvsHzgtsv(pole,b):
    [_, _, _, x, info]=scipy.linalg.lapack.zgtsv(dl2, dd2 - pole*e, du2, b)
    return x

# evaluate r(A)b in product form
t1=time.time()
ypoles = rexpi.evalr_product(mv, mvsHzgtsv, u, poles)
dt=time.time()-t1
print("time to evaluate approximation in product form %f seconds"%dt)
print("evaluate y=r(A)b in product form: \u2016y-exp(wA)b\u2016 = %.2e" % nrm(yref-ypoles))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(ypoles)-nrmb0))


# evaluate r(A)b using partial fractions
t1=time.time()
yparfrac = rexpi.evalr_partialfraction(mvsHzgtsv, u, aj, sj)
dt=time.time()-t1
print("time to evaluate approximation in partial fraction form %f seconds"%dt)
#print("max |aj| = %.2e, min Re(sj) = %.2e" % (max(np.abs(aj)),min(sj.real)))
print("evaluate y=r(A)b using partial fractions: \u2016y-exp(wA)b\u2016 = %.2e" % nrm(yref-yparfrac))
print("error in norm, |\u2016y\u2016-\u2016b\u2016| = %.2e\n"%abs(nrm(yparfrac)-nrmb0))


print("difference for product vs partial fraction: \u2016r_prod(A)b-r_PFE(A)b\u2016 = %.2e" % nrm(ypoles-yparfrac))

untiary best approximation with accuracy max|r(ix)-epx(iwx)| = 9.83e-07

time to evaluate approximation in product form 0.088788 seconds
evaluate y=r(A)b in product form: ‖y-exp(wA)b‖ = 9.18e-07
error in norm, |‖y‖-‖b‖| = 4.44e-16

time to evaluate approximation in partial fraction form 0.053865 seconds
evaluate y=r(A)b using partial fractions: ‖y-exp(wA)b‖ = 9.18e-07
error in norm, |‖y‖-‖b‖| = 1.45e-11

difference for product vs partial fraction: ‖r_prod(A)b-r_PFE(A)b‖ = 3.23e-11


## Error in inner product
Unitarity implies that the unitary best approximation conserves the inner product between two vectors within time propagation

In [9]:
print("untiary best approximation with accuracy max|r(ix)-epx(iwx)| = %.2e"%info['err'])

# construct two vectors
u1 = np.random.rand(k)+1j*np.random.rand(k)
u1 = u1/nrm(u1)
u2 = np.random.rand(k)+1j*np.random.rand(k)
u2 = u2/nrm(u2)

# inner product at initial time
ip0 = inr(u1,u2)

# time propagation (using product form) and inner product
y1 = rexpi.evalr_product(mv, mvsHzgtsv, u1, poles)
y2 = rexpi.evalr_product(mv, mvsHzgtsv, u2, poles)
ip1 = inr(y1,y2)
print("using product form |\u276Cr(A)x,r(A)y\u276D - \u276Cx,y\u276D| = %s"% abs(ip0-ip1))

# time propagation (using partial fractions) and inner product
ypfe1 = rexpi.evalr_partialfraction(mvsHzgtsv, u1, aj, sj)
ypfe2 = rexpi.evalr_partialfraction(mvsHzgtsv, u2, aj, sj)
ippfe1 = inr(ypfe1,ypfe2)

print("using partial fractions |\u276Cr(A)x,r(A)y\u276D - \u276Cx,y\u276D| = %s"% abs(ip0-ippfe1))

untiary best approximation with accuracy max|r(ix)-epx(iwx)| = 9.83e-07
using product form |❬r(A)x,r(A)y❭ - ❬x,y❭| = 2.8609792490763985e-16
using partial fractions |❬r(A)x,r(A)y❭ - ❬x,y❭| = 2.8878978388162612e-11


# Shifting the matrix spectrum

Approximate exp(tA)b assuming that an interval which contains the eigenvalues of A is known a priori

In case this interval is not $[-1,1]$ and $t=1$, then applying the unitary best approximant requires shifting and scaling the matrix to a symmetric problem first etc..

We choose the degree $n$ of the used approximant based on error estimates of the unitary best approximant and a given error tolerance.

In [139]:
# with direct solver, small dimension
nrm = lambda x : np.linalg.norm(x)
print("r(A)b \u2248 exp(wA)b where r corresponds to the unitary best approximant of degree n=%d\n"%n)
# define diagonal matrix with spectrum between -1 and 1
k=30000
print("random starting vector b with \u2016b\u2016=1, dimension k=%d\n" %k)
u = np.random.rand(k)
u = u/nrm(u)

e1 = np.ones(k-1,dtype=np.cdouble)
e = np.ones(k,dtype=np.cdouble)
o = np.zeros(k,dtype=np.cdouble)

# define a scaled and shifted 1d laplace discretization
hdiag = -1*e
hsd = e1
H = scipy.sparse.diags([hsd,hdiag,hsd], [-1,0,1])

# for small dimension check eigenvalues
# lams = scipy.linalg.eigvals(H.todense())
# print(min(lams), max(lams))

# the matrix H has eigenvalues in the interval [-3,1], define the boundaries of this interval here
l1 = -3
l2 = 1
print("assume that eigenvalues of A are located in the interval [%s,%s]\n"%(l1,l2))

# let t denote a given time-step
t = 62

# compute reference solution
yref = scipy.sparse.linalg.expm_multiply(1j*t*H,u)

print("we proceed to approximate exp(tA)b for t=%s"%t)

r(A)b ≈ exp(wA)b where r corresponds to the unitary best approximant of degree n=48

random starting vector b with ‖b‖=1, dimension k=30000

assume that eigenvalues of A are located in the interval [-3,1]

we proceed to approximate exp(tA)b for t=62


In [142]:
# choose parameters for shift and unitary best approximation
w = t*(l2-l1)/2
print("this problem requires a uniform approximation on [-w,w] for w=%f\n"%w)

tol = 1e-6
print("aim to obtain a uniform error of tol=%.2e\n"%(tol))

n = rexpi.nest(w,tol)
print("rexpi error estimate suggests using degree n=%d\n"%(n))

print("... compute unitary best approximant")
r, info = rexpi.brib(w = w, n = n, tolequi = tolequi, info=1)
print("used %d iterations, error = %.2e, deviation = %.2e\n"%(info['iterations'],info['err'],info['dev']))

print("... compute coefficients for product and partial fraction form")
a0, aj, sj = r.getpartialfractioncoef(sym=True)
poles = sj

this problem requires a uniform approximation on [-w,w] for w=124.000000

aim to obtain a uniform error of tol=1.00e-06

rexpi error estimate suggests using degree n=48

... compute unitary best approximant
used 10 iterations, error = 2.55e-07, deviation = 3.06e-04

... compute coefficients for product and partial fraction form


In [143]:
# define sparse solver and matrix-vector product for shifted matrix
# B = t*(A - i*gamma*I)/w
# relevant part of the spectrum of tA is shifted to [-w,w]
# evaluate r(itB/w) approx exp(itB)

gamma = (l1+l2)/2
mv = lambda x : 1j*t*(H.dot(x)-gamma*x)/w

# deep copies
dl2 = 1j*t*hsd/w
dd2 = 1j*t*(hdiag-gamma*e)/w
du2 = 1j*t*hsd/w
def mvsHzgtsv(pole,b):
    [_, _, _, x, info]=scipy.linalg.lapack.zgtsv(dl2, dd2 - pole*e, du2, b)
    return x

# evaluate r(A)b in product form
ypoles = rexpi.evalr_product(mv, mvsHzgtsv, u, poles)
y1 = np.exp(1j*t*gamma)*ypoles
print("evaluate y=r(tA)b, \u2016y-exp(tA)b\u2016 = %.2e" % nrm(yref-y1))


evaluate y=r(tA)b, ‖y-exp(tA)b‖ = 2.39e-07
